# Link Recommendation for Reducing Polarization in Social Networks

## Basic Classes for Calculation

In [17]:
import networkx as nx
import numpy
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 


class PolarizationGraph:
    polarization = 0

    def __init__(self, graph: nx.Graph, internal_opinions: list):
        self.nx_graph = graph
        # the internal_opinions list contains a dict for each node in the form {"value" : x, "polarization": y}
        self.internal_opinions = internal_opinions
        self.external_opinions = [
            {"value": x["value"], "polarization": 0} for x in internal_opinions]
    
    def resetExternalOpinions(self):
        for o in self.external_opinions:
            o["polarization"] = 0
    
    def addEdge(self, node_i: int, node_j: int):
        nx.add_path(self.nx_graph, [node_i, node_j])

    def setExternal(self, new_external: list):
        self.external_opinions = new_external

    def setPolarization(self, polVal: float):
        self.polarization = polVal


class PolarizationHandler:
    def calcExternalOpinions(graph: PolarizationGraph) -> list:
        expressed = []
        graph.resetExternalOpinions()

        for iter in range(1): # only one iteration to receive comparable results
            for n in graph.nx_graph.nodes():
                node_name = n if isinstance(n, int) else n[0]
                my_internal = next(
                    x["polarization"] for x in graph.internal_opinions if x["value"] == node_name)
                my_neighbours = list(nx.neighbors(graph.nx_graph, n))
                external_neighbours = [
                    x["polarization"] for x in graph.external_opinions if x["value"] in my_neighbours]
                my_expressed = (my_internal + numpy.sum(external_neighbours)) / \
                    (1 + numpy.sum([1 for neigh in my_neighbours]))

                if len(expressed) != len(graph.external_opinions):
                    expressed.append(
                        {"value": node_name, "polarization": my_expressed})
                else:
                    my_set = next(
                        s for s in expressed if s["value"] == node_name)
                    my_set["polarization"] = my_expressed

            graph.setExternal(expressed)
            # print(PolarizationHandler.calcPolarization(graph))

        return expressed

    def calcPolarization(graph: PolarizationGraph) -> float:
        polArr = [x["polarization"] for x in graph.external_opinions]
        nbrNodes = nx.number_of_nodes(graph.nx_graph)
        return numpy.linalg.norm(polArr) / nbrNodes
    
    def initPolarization(graph:PolarizationGraph):
        PolarizationHandler.calcExternalOpinions(graph)
        pol = PolarizationHandler.calcPolarization(graph)
        graph.setPolarization(pol)

class HeuristicsHandler:
    # this class gives us the target nodes whose polarization we want to reduce by new edges
    # we still need to find the other part of the edge

    def getExtremeExpressed(graph: PolarizationGraph, nbr_of_nodes_to_neutralize=None) -> list:
        external_opinions = graph.external_opinions
        
        sorted_list = sorted(external_opinions, key=lambda x: x["polarization"], reverse=True)
        
        if nbr_of_nodes_to_neutralize is None:
            return sorted_list
        else:
            return sorted_list[0:nbr_of_nodes_to_neutralize]
        
    def getExtremeNeighbours(graph: PolarizationGraph, nbr_extreme_neighborhoods = None) -> list:
        neighbourhood_extremities = []
        for n in graph.nx_graph.nodes():
            n_neighbours = list(nx.neighbors(graph.nx_graph, n))
            external_neighbours = [x["polarization"] for x in graph.external_opinions if x["value"] in n_neighbours]
            n_neighbourhood_extremity = sum(map(abs, external_neighbours))
            n_dict = {"value": n, "neighbourhood_extremity": n_neighbourhood_extremity}
            neighbourhood_extremities.append(n_dict)
        
        neighbourhood_extremities = sorted(neighbourhood_extremities, key=lambda x: x["neighbourhood_extremity"], reverse=True)
        
        if nbr_extreme_neighborhoods is None:
            return neighbourhood_extremities
        else:
            return neighbourhood_extremities[0:nbr_extreme_neighborhoods]
    
    def getPagerank(graph: PolarizationGraph, nbr_pageranks = None) -> dict:
        graph_pagerank = nx.pagerank(graph.nx_graph)
        graph_pagerank = [{"value": node, "pagerank": pr} for node, pr in sorted(graph_pagerank.items(), key=lambda item: item[1], reverse=True)]
        print(graph_pagerank)
        return graph_pagerank

class LinkRecommender:
    # HeuristicsHandler gives us i in edge (i,j) we want to create
    # idea for j: iterate a list that is sorted from highest opposite opinion
    # and get the link probability for each link between (i,j)
    # take j when the link probability is higher than a certain threshold WHICH WE NEED TO DEFINE
    # To define the threshold, we initialize it with 0, compare polarization, increase the threshold, compare polarization, etc

    # Shall we use those algorithms to receive the link probability? https://networkx.org/documentation/stable/reference/algorithms/link_prediction.html

    def extremeExpressedAndCommonNeighbors(graph: PolarizationGraph, k=None) -> list:
        nodesExtreme = HeuristicsHandler.getExtremeExpressed(graph, k)
        return LinkRecommender.getEdgesByCommonNeighbors(graph, nodesExtreme)

    def extremeNeighborsAndCommonNeighbors(graph: PolarizationGraph, k = None) -> list:
        nodesExtremeNeighb = HeuristicsHandler.getExtremeNeighbours(graph, k)
        return LinkRecommender.getEdgesByCommonNeighbors(graph, nodesExtremeNeighb)
    
    def pagerankAndCommonNeighbors(graph: PolarizationGraph, k = None) -> list:
        nodes = HeuristicsHandler.getPagerank(graph, k)
        return LinkRecommender.getEdgesByCommonNeighbors(graph, nodes)

    def getEdgesByCommonNeighbors(graph: PolarizationGraph, investigatedNodes: list) -> list:
        recommendedEdges = []

        for n in investigatedNodes:
            my_node = n["value"]
            
            no_neighbors = list(nx.non_neighbors(graph.nx_graph, my_node))
            possible_nodes = [x for x in graph.external_opinions if x["value"] in no_neighbors]

            # if n is negatively polarized, then look first at the most positive polarized nodes
            reverseNeeded = next(x["polarization"] < 0 for x in graph.external_opinions if x["value"] == n["value"] )
            sortedNodesByPolarization = sorted(possible_nodes, key=lambda x: x["polarization"], reverse=reverseNeeded)

            # get amount of common neighbours 
            for m in sortedNodesByPolarization:
                current_common_neighbors = list(nx.common_neighbors(graph.nx_graph, my_node, m["value"]))
                m["nbr_common_neighbors"] = len(current_common_neighbors)
                m["weighted_polarization"] = m["nbr_common_neighbors"] * m["polarization"]

            sortedNodesByWeight = sorted(sortedNodesByPolarization, key=lambda x: x["weighted_polarization"], reverse=reverseNeeded)
            winnerNode = sortedNodesByWeight[0]
            recommendedEdges.append([my_node, winnerNode["value"]])

        return recommendedEdges









# Data Setup

In [18]:
import networkx as nx
#import matplotlib.pyplot as plt

KarateGraph = nx.read_gml("karate.gml", None)
BooksGraph = nx.read_gml("polbooks.gml", None)

ground_truth_as_sets = [{20, 1, 2, 22, 3, 5, 4, 7, 6, 8, 11, 13, 12, 14, 18, 10, 17},
                        {24, 26, 28, 33, 30, 34, 25, 32, 27, 21, 23, 29, 9, 31, 15, 16, 19}]

internal_karate = []
internal_books = []

for i in range(1, 35):
    polarization = -1 if i in ground_truth_as_sets[0] else 1
    internal_karate.append({"value": i, "polarization": polarization})

for n in BooksGraph.nodes.data():
    polarization = 0 if n[1]["value"] == "n" else - \
        1 if n[1]["value"] == "c" else 1
    internal_books.append({"value": n[0], "polarization": polarization})

KaratePolGraph = PolarizationGraph(KarateGraph, internal_karate)
BooksPolGraph = PolarizationGraph(BooksGraph, internal_books)

x = range(100)


for gr in [
    KaratePolGraph, 
    BooksPolGraph
]:
    y = []
    PolarizationHandler.initPolarization(gr)
    print("Initial Polarization: " + str(gr.polarization))

    for e_index in range(1,101):
        #newEdges = LinkRecommender.extremeNeighborsAndCommonNeighbors(gr, 1)
        newEdges = LinkRecommender.extremeExpressedAndCommonNeighbors(gr, 1)
        #newEdges = LinkRecommender.pagerankAndCommonNeighbors(gr, 1)


        for e in newEdges:
            nx.add_path(gr.nx_graph, [e[0], e[1]])
            
        PolarizationHandler.initPolarization(gr)
        y.append(gr.polarization)

        print(f'Polarization after adding {e_index} new link(s): {gr.polarization}, added edge from {e[0]} to {e[1]}')
    
    #fig, ax = plt.subplots()  # Create a figure containing a single axes.
    #ax.plot(x, y)  # Plot some data on the axes.
    #plt.show()





Initial Polarization: 0.044040129270708835
[{'value': 34, 'pagerank': 0.1009179167487121}, {'value': 1, 'pagerank': 0.09700181758983706}, {'value': 33, 'pagerank': 0.07169213006588289}, {'value': 3, 'pagerank': 0.05707842304763673}, {'value': 2, 'pagerank': 0.052878391037427}, {'value': 32, 'pagerank': 0.037156635922679405}, {'value': 4, 'pagerank': 0.035860643223064786}, {'value': 24, 'pagerank': 0.03152091531163227}, {'value': 9, 'pagerank': 0.029765339186167028}, {'value': 14, 'pagerank': 0.02953631497720298}, {'value': 6, 'pagerank': 0.02911334166344221}, {'value': 7, 'pagerank': 0.029113341663442205}, {'value': 30, 'pagerank': 0.026287262837112076}, {'value': 28, 'pagerank': 0.025638803528350497}, {'value': 31, 'pagerank': 0.024589336534292478}, {'value': 8, 'pagerank': 0.02449075803950918}, {'value': 5, 'pagerank': 0.021979406974834498}, {'value': 11, 'pagerank': 0.021979406974834494}, {'value': 25, 'pagerank': 0.021075455001162945}, {'value': 26, 'pagerank': 0.02100562817474579}

IndexError: list index out of range